# Model HALVES -- Conditional Trials
## Compares periods of learning and nonlearning between early (run1) and late (run2)

In [3]:
%matplotlib inline 
import os
from os.path import join, split
import numpy as np
import pandas as pd
from scipy import stats
from glob import glob


subs = ['WMAZE_001', 'WMAZE_002', 'WMAZE_004', 'WMAZE_005', 'WMAZE_006',
        'WMAZE_007', 'WMAZE_008', 'WMAZE_009', 'WMAZE_010', 'WMAZE_012',
        'WMAZE_017', 'WMAZE_018', 'WMAZE_019', 'WMAZE_020', 'WMAZE_021',
        'WMAZE_022', 'WMAZE_023', 'WMAZE_024', 'WMAZE_026', 'WMAZE_027']
sets = ['set1', 'set2', 'set3']


##STEP 1
#iterate through subjects
for sub in subs:
    sub_dir = '/home/data/madlab/Mattfeld_WMAZE/sourcedata/behav/{0}'.format(sub) #base directory 
    save_dir = join(sub_dir, 'model_HALVES_C/')
    
    #create directory for save data
    if not os.path.exists(save_dir): 
        os.makedirs(save_dir)
    
    #grab the derivative files created from the *set-based* learning analysis and sort
    frst_deriv_files = sorted(glob(join(sub_dir, 'Bprime_pmode_set*.txt')))     
    learning_files = sorted(glob(join(sub_dir, 'B_pmode_set*.txt')))     
    behav_runs = sorted(glob(join(sub_dir, '{0}_wmazebl_2015*.txt'.format(sub))))
    
    
    ##STEP 2
    #iterate through stimulus sets
    for i, curr_set in enumerate(sets): #load derivative, learning curve, and behavioral files
        deriv_file = np.loadtxt(frst_deriv_files[i])
        learning_curve = np.loadtxt(learning_files[i]) 
        #grab two runs for each stimulus set
        behav_run1 = pd.read_table(behav_runs[i*2])
        behav_run2 = pd.read_table(behav_runs[i*2+1])
                
        #info concerning onset time
        behav_os = np.empty(320, dtype=object)
        behav_os[:160] = behav_run1['StimOnset'].values
        behav_os[160:] = behav_run2['StimOnset'].values + (197*2) #number of volumes x TR 
        #info concerning subject response
        behav_resp = np.empty(320, dtype=object)
        behav_resp[:160] = behav_run1['Resp'].values
        behav_resp[160:] = behav_run2['Resp'].values        
        #info concerning trial type
        behav_type = np.empty(320, dtype=object)
        behav_type[:160] = behav_run1['TrialType'].values
        behav_type[160:] = behav_run2['TrialType'].values
        
        
        ##STEP 3
        #identify position (index) of conditional trials
        b_indices = np.where((behav_type=='B'))[0] #Bs in the original dataset without preceeding BL trials
        trial_shift = behav_type[:-1] #remove last trial in behav_type
        trial_shift = np.insert(trial_shift,0,-1) #shifted down to identify BL --> B (places "-1" at 0 index)
        b_BL_indices = np.where((behav_type=='B')&(trial_shift=='BL'))[0] #B trials with preceeding BLs
        trial_shift2 = behav_type[1:] 
        trial_shift2 = np.append(trial_shift2,-1) #shifted up to grab anything after B (places "-1" at last index)
        #identify conditional, baseline and nonresponse trials
        most_remaining = np.where((behav_type =='B')|(behav_type=='BL')|((trial_shift2!='B')
                                  &((behav_type=='A')|(behav_type=='C'))))[0]

        
        ##STEP 4
        #isolate bad Bs - value represents index among all trials in set [0:360] 
        bad_Bs = [] 
        bad_Bs.extend(b_BL_indices)
        for curr_B in b_indices:
            if not curr_B in bad_Bs:
                if behav_resp[curr_B] == 'NR': 
                    bad_Bs.append(curr_B)    
                if curr_B in [0, 157, 158, 159, 160, 317, 318, 319]:  
                    bad_Bs.append(curr_B)
        bad_Bs.sort() 
             
            
        ##STEP 5         
        #obtains B-specific array indices [0:80] to match length of curve and derivative files 
        bad_B_ind = sorted([j for j, f in enumerate(b_indices) if f in bad_Bs]) #get bad Bs within Bs 
        bad_B_ind = bad_B_ind[::-1] #reverse order of Bs to be removed (required for .pop()) 
        
        
        ##Step 6
        #add fixed before bad Bs to create complete all_remaining
        nonBL_bad_Bs = [f for f in bad_Bs if f not in b_BL_indices] #check for bad conditionals not preceded by BLs
        fixed_bads = np.array([x - 1 for x in nonBL_bad_Bs]) #subtract one from each index, convert into numpy array
        all_remaining = np.concatenate((most_remaining, fixed_bads)) #merge fixed_bads and most_remaining
        all_remaining = [int(f) for f in all_remaining] #make sure all arrays are integers (WMAZE_007 is not)
      
        
        ##STEP 7
        #remove bad conditional trials from learning curve
        temp = list(learning_curve) #temp version of learning_curve
        for curr_bad_B in bad_B_ind: 
            temp.pop(curr_bad_B) #pop out the bad Bs starting from the end   
        new_learning = np.array(temp) #save without the removed Bs
     
    
        ##STEP 8
        #remove bad conditional trials from derivative
        temp1 = list(deriv_file)
        for curr_bad_B in bad_B_ind:
            temp1.pop(curr_bad_B)  
        new_deriv = np.array(temp1[1:]) #remove 1st values in derivative file (0)      
        learning = np.where(new_deriv > 0)[0] #values greater than 0 == "learning"
        nonlearning = np.where(new_deriv <= 0)[0] #values <= 0 == "nonlearning"

        
        ##STEP 9
        #use B-specific indices [0:80] to remove bad conditionals from original array of B indices [0:360]
        temp2 = list(b_indices)
        for curr_bad_B in bad_B_ind:
            temp2.pop(curr_bad_B)
        good_Bs = np.array(temp2)
                
        
        ##STEP 10
        #index good conditional indices using the learning/nonlearning indices
        learn_ind = good_Bs[learning]
        nonlearn_ind = good_Bs[nonlearning]
    
        
        ##STEP 11
        #use list comprehension to grab values matching index criteria for Run1 and Run2
        run1_learn = [f for f in learn_ind if f <= 156]
        run1_nonlearn = [f for f in nonlearn_ind if f <= 156]       
        run2_learn = [f for f in learn_ind if f > 160]
        run2_nonlearn = [f for f in nonlearn_ind if f > 160]
        
        
        ##Step 12

        #index behavioral onset array using fixed trial indices
        run1_learn_os = behav_os[run1_learn]
        run1_nonlearn_os = behav_os[run1_nonlearn]
        run2_learn_os = behav_os[run2_learn]
        run2_nonlearn_os = behav_os[run2_nonlearn]
        all_remaining_os = behav_os[all_remaining]
        
        
        ##STEP 13
        #create matrices in the format required by first level Bunch()
        early_learn_mtrx = np.vstack((run1_learn_os,np.ones(len(run1_learn_os))*2.5, 
                                     np.ones(len(run1_learn_os)))).T                
        early_nonlearn_mtrx = np.vstack((run1_nonlearn_os,np.ones(len(run1_nonlearn_os))*2.5,
                                        np.ones(len(run1_nonlearn_os)))).T        
        late_learn_mtrx = np.vstack((run2_learn_os,np.ones(len(run2_learn_os))*2.5, 
                                     np.ones(len(run2_learn_os)))).T       
        late_nonlearn_mtrx = np.vstack((run2_nonlearn_os,np.ones(len(run2_nonlearn_os))*2.5,
                                        np.ones(len(run2_nonlearn_os)))).T       
        all_remaining_mtrx = np.vstack((all_remaining_os,np.ones(len(all_remaining_os))*2.5, 
                                        np.ones(len(all_remaining_os)))).T
        
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)        
        #write EVs to .txt file
        for trial in ['early_learn', 'early_nonlearn', 'late_learn', 'late_nonlearn', 'all_remaining']: 
            exec('np.savetxt(save_dir + "{0}_{1}.txt", {1}_mtrx, delimiter="\t", fmt="%.4f")'.format(curr_set,trial))